In [1]:
from qiskit_metal import draw, Dict
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.base.base import QComponent
import qiskit_metal as metal

In [2]:
design = metal.designs.DesignPlanar()

In [3]:
import numpy as np
from math import *
from qiskit_metal import draw, Dict
from qiskit_metal.qlibrary.base.qubit import BaseQubit

In [4]:
class SeeqcTrasmon(BaseQubit):
    """
    """

    default_options = Dict(
        width='250um',  # width of transmon pocket
        height='250um', # height of transmon pocket
        layer='50',     # GDS layer
        radius='60um',  # radius of qubit body
        gap='20um',     # gap from qubit to ground
        jj_w='10um',    # JJ width
        cap_w='2um',    # coupling capacitor width
        cap_ang='60',# coupling cap angle

        pocket_w='500um',
        pocket_h='500um',
        position_x='0.0mm',
        position_y='0.0mm',
        rotation='0.0',
        cpw_width='5.0um'# width of CPW
    )
    """Default drawing options"""

    component_metadata = Dict(short_name='q',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True'
    )
    """Component metadata"""

    def make(self):
        """Convert self.options into QGeometry."""

        p = self.parse_options()    # Parse string options into numbers

        # Draw the pad
        qubit_pad = draw.Point(0,0).buffer(p.radius)
        space = draw.Point(0,0).buffer(p.gap + p.radius)
        gap = draw.subtract(space, qubit_pad)

        qubit_etch = qubit_pad.buffer(p.gap)

        # Draw coupling capacitors
        npts = 64
        cap_radius = p.radius + p.gap/2
        
        caps = []

        for ang in np.pi * np.arange(1/4, 2, 1/2):

            cap_thetas = np.pi * p.cap_ang * np.linspace(-0.5, 0.5, npts) / 180
            cap_pts = np.column_stack([
                cap_radius * np.cos(cap_thetas + ang),
                cap_radius * np.sin(cap_thetas + ang)])
            cap_line = draw.LineString(cap_pts)
            cap = cap_line.buffer(p.cap_w, cap_style=2)

            # Stub
            stub = draw.LineString

            caps.append(cap)

        # Add Junction/SQUID
        rect_jj = draw.LineString([(0,-p.radius),
                                    (0, -(p.radius + p.gap))

        ])

        objects = [qubit_pad, qubit_etch, rect_jj, *caps]
        objects = draw.rotate(objects, p.rotation, origin=(0,0))
        objects = draw.translate(objects, xoff=p.position_x, yoff=p.position_y)
        [qubit_pad, qubit_etch, rect_jj, c1, c2, c3, c4] = objects

        # Use the geometry to create Metal QGeometry

        self.add_qgeometry('poly', dict(qubit_pad=qubit_pad), subtract=False)
        self.add_qgeometry('poly', dict(qubit_etch=qubit_etch), subtract=True)

        self.add_qgeometry('poly', dict(c1=c1), subtract=False)
        self.add_qgeometry('poly', dict(c2=c2), subtract=False)
        self.add_qgeometry('poly', dict(c3=c3), subtract=False)
        self.add_qgeometry('poly', dict(c4=c4), subtract=False)

        self.add_qgeometry('junction', dict(rect_jj=rect_jj), width=p.jj_w)



## Test placement

In [5]:
gui = metal.MetalGUI(design)

In [6]:
q1 = SeeqcTrasmon(design,'my_transmon',options=dict(connection_pads=dict(a=dict(),b=dict(loc_W=-1))))
gui.rebuild()